In [31]:
from dk_model import DeepKrigingTrainer
import pandas as pd 

import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from pykrige.ok import OrdinaryKriging
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [32]:
deposit_data = pd.read_csv("../Curated_data/final_dataset_1_no_0.csv", low_memory=False)


In [33]:
total_columns = ['CP_Total','PO_Total', 'PY_Total']

#all covariates
covariates = total_columns[:3] + ['RQD_Pct', 'Cr_ppm'] 

In [34]:
phi_columns = deposit_data.columns[10:].tolist()


# 1. Deepkriging and DNN

In [35]:
print("Metrics for Deepkriging")
trainer = DeepKrigingTrainer(deposit_data, regular_nn=False, plot_errors=False)
trainer.train_neural_network()

print("\nMetrics for regular DNN")
trainer = DeepKrigingTrainer(deposit_data, regular_nn=True, plot_errors=False)
trainer.train_neural_network()

print("\nMetrics for Deepkriging with covariates")
trainer = DeepKrigingTrainer(deposit_data, covariates, regular_nn=False, plot_errors=False)
trainer.train_neural_network()

print("\nMetrics for regular DNN with covariates")
trainer = DeepKrigingTrainer(deposit_data, covariates, regular_nn=True, plot_errors=False)
trainer.train_neural_network()


Metrics for Deepkriging

Average Metrics Across Folds:
  Average MSE: 0.0030
  Average MAE: 0.0395
  Average adjusted R2: 1.2264
  STD MSE: 0.0004
  STD MAE: 0.0025
  STD adjusted R2: 0.0539

Metrics for regular DNN

Average Metrics Across Folds:
  Average MSE: 0.0078
  Average MAE: 0.0573
  Average adjusted R2: 0.4918
  STD MSE: 0.0027
  STD MAE: 0.0067
  STD adjusted R2: 0.1745

Metrics for Deepkriging with covariates

Average Metrics Across Folds:
  Average MSE: 0.0026
  Average MAE: 0.0353
  Average adjusted R2: 1.1925
  STD MSE: 0.0006
  STD MAE: 0.0034
  STD adjusted R2: 0.0620

Metrics for regular DNN with covariates

Average Metrics Across Folds:
  Average MSE: 0.0066
  Average MAE: 0.0534
  Average adjusted R2: 0.5575
  STD MSE: 0.0009
  STD MAE: 0.0024
  STD adjusted R2: 0.0804


# 2. Reg kriging 

In [36]:
np.random.seed(42)


y = deposit_data['Density_gcm3'].values[:, np.newaxis]  # Keep variable as the output
x = deposit_data[['X', 'Y', 'Z','CP_Total', 'PO_Total', 'PY_Total', 'RQD_Pct', 'Cr_ppm']].values
nvars = 8
x = x.reshape(len(deposit_data), nvars)

num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)


mse_list, mae_list, adjusted_r2_list = [], [], []

for train_index, test_index in kf.split(x):
    X_cv_train, X_cv_test = x[train_index], x[test_index]
    y_cv_train, y_cv_test = y[train_index], y[test_index]

    # Fit a linear regression model
    regression_model = LinearRegression()
    regression_model.fit(X_cv_train, y_cv_train)

    # Predictions from the regression model
    y_cv_pred = regression_model.predict(X_cv_test)

    # Residuals (the difference between actual and predicted values)
    residuals = y_cv_test - y_cv_pred

    # Ordinary Kriging on residuals
    ok = OrdinaryKriging(X_cv_test[:, 0], X_cv_test[:, 1], residuals, variogram_model='linear', verbose=False)
    kriging_pred, _ = ok.execute('grid', X_cv_test[:, 0], X_cv_test[:, 1])


    # Combine regression predictions with kriging predictions
    final_cv_predictions = y_cv_pred + kriging_pred

    # Calculate and store metrics
    mse = np.mean((y_cv_test - final_cv_predictions) ** 2)
    mae = np.mean(np.abs(y_cv_test - final_cv_predictions))
    
 #   sst = np.mean((y_cv_test - np.mean(y_cv_test)) ** 2) * len(y_cv_test)
 #   ssr = np.mean((final_cv_predictions - y_cv_test) ** 2) * len(y_cv_test)
    sst = np.mean((y_cv_test - np.mean(y_cv_test)) ** 2) 
    ssr = np.mean((final_cv_predictions - y_cv_test) ** 2) 
    r2 = 1 - (ssr / sst)
    
    # Calculate adjusted R-squared
    n = len(y_cv_test)
    num_predictors = X_cv_test.shape[1]
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - num_predictors - 1))

    
    mse_list.append(mse)
    mae_list.append(mae)
    adjusted_r2_list.append(adjusted_r2)

# Calculate mean metrics across folds


# Print average metrics across folds
print("Metrics for regression kriging with covariates")
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(mse_list):.4f}")
print(f"  Average MAE: {np.mean(mae_list):.4f}")
print(f"  Average adjusted R2: {np.mean(adjusted_r2_list):.4f}")



Metrics for regression kriging with covariates

Average Metrics Across Folds:
  Average MSE: 0.0136
  Average MAE: 0.0751
  Average adjusted R2: 0.1016


In [37]:
np.random.seed(42)

y = deposit_data['Density_gcm3'].values[:, np.newaxis]  # Keep variable as the output
x = deposit_data[['X', 'Y', 'Z']].values
x = x.reshape(len(deposit_data), 3)

mse_list = []
mae_list = []
test_adjusted_r2_list = []

num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for train_index, test_index in kf.split(x):
    X_cv_train, X_cv_test = x[train_index], x[test_index]
    y_cv_train, y_cv_test = y[train_index], y[test_index]

    # Ordinary Kriging on residuals
    ok = OrdinaryKriging(X_cv_test[:, 0], X_cv_test[:, 1], y_cv_test, variogram_model='linear', verbose=False)
    kriging_pred, _ = ok.execute('grid', X_cv_test[:, 0], X_cv_test[:, 1])

    final_cv_predictions =  kriging_pred

    # Calculate and store metrics
    mse = np.mean((y_cv_test - final_cv_predictions) ** 2)
    mae = np.mean(np.abs(y_cv_test - final_cv_predictions))
    mse_list.append(mse)
    mae_list.append(mae)

    n = len(y_cv_test)
    sst = np.mean((y_cv_test - np.mean(y_cv_test)) ** 2)
    ssr = np.mean((final_cv_predictions - y_cv_test) ** 2) 
    r2 = 1 - (ssr / sst)

    
    num_predictors = X_cv_test.shape[1]

    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - num_predictors - 1))
    test_adjusted_r2_list.append(adjusted_r2)

# Calculate mean metrics across folds
mean_mse = np.mean(mse_list)
mean_mae = np.mean(mae_list)
mean_a_r2 = np.mean(test_adjusted_r2_list)

# Print average metrics across folds
print("Metrics for kriging")
print(f"  Average MSE: {mean_mse:.5f}")
print(f"  Average MAE: {mean_mae:.5f}")
print(f"  Average adjusted R2: {mean_a_r2:.5f}")


Metrics for kriging
  Average MSE: 0.01437
  Average MAE: 0.07750
  Average adjusted R2: 0.08149
